In [ ]:
#!pip install pandas

In [16]:
# Bibliotecas
from osgeo import gdal, ogr
import glob
import numpy as np
import re
from datetime import datetime
import pandas as pd
import os
import xml.etree.ElementTree as ET
from datetime import datetime


In [11]:
def create_mask_from_vector_per_polygon(vector_data_path, cols, rows, geo_transform,
                            projection, attribute = 'id', all_touched= False, nodatavalue=-9999):
    #"""Rasterize the given vector (wrapper for gdal.RasterizeLayer).""" # Funcion obtenida de internet y modififcada
    """ Rasteriza una capa vectorial

    Parameters
    ----------
    vector_data_path: str
      Direccion del archivo del shapefile
    cols: int
      Cantidad de columnas deseadas para el raster
    rows: int
     Cantidad de filas deseadas para el raster
    geo_transform: 
      
    projection:
    
    attribute: str
      Nombre de la columna del shapefile que se utilizará para identificar a los polígonos
    all_touched: bool
      Si es True, se consideraran todos los pixeles que intersecan con los polígono. Si es False, solo se consideraran aquellos pixeles que caen dentro del poligono.
    nodatavalue=-9999
 
    Returns
    -------
    np.array
        un array que tendra valores 0 en aquellos lugares donde no haya poligonos y valores del attribute para cada poligono.
    """

    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)

    driver = gdal.GetDriverByName('GTiff')  # In memory dataset
    target_ds = driver.Create('target.tif', cols, rows, 1, gdal.GDT_Float32)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)

    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(nodatavalue)

    if all_touched ==True:
    	gdal.RasterizeLayer(target_ds, [1], layer, options=["ATTRIBUTE="+attribute, "ALL_TOUCHED= TRUE"])
    else:
    	gdal.RasterizeLayer(target_ds, [1], layer, options=["ATTRIBUTE="+attribute])
    return target_ds.ReadAsArray()


def get_saocom_date(src_xemt_file):
  """ Dado un arhivo xemt, obtiene la fecha de adquisicion

    Parameters
    ----------
    src_xemt_file : str
        Direccion al archivo xemt

    Returns
    -------
    str
        Fecha de adquisicion en formato:
  """

  tree = ET.parse(src_xemt_file)
  root = tree.getroot()
  s =ET.tostring(root, encoding='utf8').decode('utf8')
  substring_acq = s[s.find('<acquisitionTime>'):s.find('</acquisitionTime>')]
  n =substring_acq.find('<startTime>')
  return substring_acq[n+11:n+21]

def get_date(raster_name_src, sensor, alos_dates_csv = '/media/nmorandeira/T7/Img_preprocesadas/Raster/SAR/ALOS/asf-datapool-index.csv', saocom_dates_xmet=''):
  """ Para cada sensor (ALOS, SENTINEL1, SAOCOM) obtiene un string que representa las fechas de adquisicion de la imagen

    Parameters
    ----------
    raster_name_src : str
        Direccion al archivo raster de la imagen
    sensor: str
        Detalle del sensor. Opciones: ALOS, SENTINEL1, SAOCOM
    alos_dates_csv: str
        Direccion al csv que contiene las fechas de las imagenes
    saocom_dates_xmet: str
        Direccion al xmet que contiene las fecha de la imagene
    Returns
    -------
    str
        Fecha de adquisicion en formato:
  """
  if sensor == 'SENTINEL':
    result = re.search('1SDV_(.*)', raster_name_src)
    result = datetime.strptime(result.group(1)[:8], '%Y%m%d').strftime('%Y-%m-%d')#.strftime('%d/%m/%Y')
  elif sensor== 'ALOS':
    n = raster_name_src.find('ALPSRP')
    id_img_name = raster_name_src[n:n+15]
    df_alos_dates = pd.read_csv(alos_dates_csv)
    result = df_alos_dates[df_alos_dates['Granule Name']==id_img_name]['Acquisition Date'].values[0][:10]
  elif sensor== 'SAOCOM':
    result = get_saocom_date(saocom_dates_xmet)
  return result


def vector_table_to_dic(shp_file, replace_spaces_in_strings = True):
  """Crea un diccionario a partir de los valores de la tabla del shapefile

  Parameters
  ----------
  shp_file : str
      Direccion al archivo shapefile
  replace_spaces_in_strings : bool, optional
      Si es True, reemplaza los espacios que aparecen en la tabla por "_"

  Returns
  -------
  dict
      Diccionario
  """
  
  dataSource = ogr.Open(shp_file) #abrir el shapefile como objeto espacial
  daLayer = dataSource.GetLayer(0) #obtener el primer ítem del objeto espacial
  layerDefinition = daLayer.GetLayerDefn() #obtener info de campos
  l = []
  for i in range(layerDefinition.GetFieldCount()):
      l.append(layerDefinition.GetFieldDefn(i).GetName())
  d = {}
  i = 0
  for feature in daLayer:
    d[i] = {}
    for field in l:
      d[i][field] = feature.GetField(field)
      #print(field, feature.GetField(field))
    i = i+1

  if replace_spaces_in_strings:
    for k in d.keys():
      for j in d[k].keys():
        if type(d[k][j])==str:
          d[k][j]=d[k][j].replace(' ', '_')
  return d

def add_table_values_to_df(d,df):
  """Pasa los datos contenidos en un diccionario d a un pd.DataFrame df

    Parameters
    ----------
    d : dict

    df : pd.DataFrame


    Returns
    -------
    DataFrame
        
  """
  ids = [int(d[k]['id']) for k in d.keys()]
  for i in ids:
    values = [k for k in d.keys() if d[k]['id']==i]
    d_ = d[values[0]]
    for k in d_.keys():
      df.loc[df['id'] == i, k] = d_[k]
  return df

def get_bands_information(src):
  """Obtiene el level de imagen, la canidad de bandas y los nombres de cada una de ellas.

    Parameters
    ----------
    src : str
        Direccion al archivo (bandas.txt) que contiene la informacion de las bandas de la imagen

    Returns
    -------
    level
        un string que representa el level de la imagen
    n_bandas
        un int que representa la cantidad de bandas que contiene la imagen
    bandas_nombres
        una lista que contiene los nombres de las bandas (str) de manera ordenada
  """
  with open(src) as f:
    lines = f.readlines()
  level = lines[0].replace('\n',"")
  n_bandas = len(lines)-1
  bandas_nombres = []
  for v in range(1,n_bandas+1):
    bandas_nombres.append(lines[v].replace('\n',""))
  
  return level, n_bandas, bandas_nombres

def get_mean_std_values_from_img(img_complete, shp_file, n_bandas, bandas_nombres):
  img_test = gdal.Open(img_complete,gdal.GA_ReadOnly)
  img_test_arr = img_test.ReadAsArray()
  #print(img_test_arr.shape)

  rows, cols = img_test_arr.shape[1:]
  geo_transform = img_test.GetGeoTransform()
  projection = img_test.GetProjection()

  target = create_mask_from_vector_per_polygon(shp_file, cols, rows, geo_transform, projection, attribute = 'id', all_touched= False)
  a, c = np.unique(target, return_counts = True)
  d = {}
  nodatavalue=-9999
  a = [i for i in a if i!=nodatavalue]

  for i in a:
    mask = target == i
    np.unique(mask, return_counts=True)
    masked = img_test_arr[:,mask[:,:]]
    if n_bandas>1:
      means = np.mean(masked, axis=1)
      stds = np.std(masked, axis=1)
    d_aux = {}

    incidence_angle_band = np.where(np.array(bandas_nombres)=='incidence_angle' )[0][0]
    incidence_angle_mean = means[incidence_angle_band]

    d_calculated = {}
    
    if np.where(np.array(bandas_nombres)=='HV2')[0].shape[0]!=0:
      hv2_band_num = np.where(np.array(bandas_nombres)=='HV2')[0][0]
      d_calculated['HV'] = img_test_arr[hv2_band_num,:,:]/2

    if sum([i.find('C13')!=-1 for i in bandas_nombres])==2:
      c13real_band_num = np.where(np.array(bandas_nombres)=='C13_real')[0][0]
      c13imag_band_num = np.where(np.array(bandas_nombres)=='C13_imag')[0][0]

      c13real =  img_test_arr[c13real_band_num, :,:] 
      c13imag =  img_test_arr[c13imag_band_num, :,:] 
      # Modificado 10.06.2022
      np.seterr(divide='ignore', invalid='ignore')
      c13imag = c13imag.astype('float')
      d_calculated['CPD'] = np.degrees(np.arctan(c13real/c13imag))

    for b in [x for x in range(n_bandas) if x != incidence_angle_band]:
      banda_nombre = bandas_nombres[b]
      if banda_nombre!='HV2' and banda_nombre!='C13_real' and banda_nombre!='C13_imag':
        mean = means[b]
        std = stds[b]
      elif banda_nombre == 'HV2':
        mean = np.mean(d_calculated['HV'][mask[:,:]])
        std = np.std(d_calculated['HV'][mask[:,:]])
        banda_nombre = 'HV'
        #print(banda_nombre, mean, std)
      elif banda_nombre == 'C13_real':
        mean = np.mean(d_calculated['CPD'][mask[:,:]])
        std = np.std(d_calculated['CPD'][mask[:,:]])
        banda_nombre = 'CPD'
        #print(banda_nombre, mean, std)

      if mean!=0 and std!=0 and banda_nombre!='C13_imag' and ~np.isnan(mean) and ~np.isnan(std):# aca camboar para que sean ~(mean==0 and std==0)
        #print(banda_nombre)
        #d_aux[b] = {'id':int(i), 'banda_num':b, 'banda_nombre':banda_nombre,'valor_promedio': mean, 'valor_desvioest': std, 'angulo_incidencia':incidence_angle_mean}
        d_aux[b] = {'id':int(i), 'banda_num':b, 'banda_nombre':banda_nombre, 'angulo_incidencia':incidence_angle_mean,'valor_promedio': mean, 'valor_desvioest': std}
    if d_aux !={}:
      d[i] = d_aux  
  return d

def get_csv(sensor_and_level, shp_file, dataframes_folder, sensor_name_sat, lista_de_imagenes_no_procesadas=[]):
  bands_file = sensor_and_level + '/bands.txt'
  imgs_list = glob.glob(sensor_and_level+'/*.tif')

  level, n_bandas, bandas_nombres = get_bands_information(bands_file)

  for img_complete in imgs_list:

    print('---------------------------------------')
    print('Corriendo: ', img_complete)

    saocom_dates_xmet = None
    if sensor_name_sat == 'SAOCOM':
      xemt_files = glob.glob('/media/nmorandeira/T7/ExtraccionSAR/SAOCOM_shifted/Raster/SAR/SAOCOM/SAOCOM_StripMap_QP/xemt/*xemt')
      #saocom_dates_xmet = [i for i in xemt_files if i.find(img_complete[img_complete.find('S1'): img_complete.find('_mat')])!=-1][0]
      saocom_dates_xmet = [i for i in xemt_files if i.find(img_complete[img_complete.find('S1'): img_complete.find('S1')+48])!=-1][0]


    # Verifico que el txt y la imagen sean congruentes:
    img_bandas = gdal.Open(img_complete).ReadAsArray().shape[0]
    if n_bandas != img_bandas:
      #loggin. ----- pasar lo siguiente a log y break/continue
      print('Las bandas del txt y de la imagen'+img_complete+' no coinciden.')
      lista_de_imagenes_no_procesadas.append(img_complete)
      continue
    else:
      # values = get_mean_std_values_from_img_v2(img, shp_file, n_bandas)
      # print(img_complete, shp_file, n_bandas, bandas_nombres)
      values = get_mean_std_values_from_img(img_complete, shp_file, n_bandas, bandas_nombres)
      for v in values.keys():
        df_aux = pd.DataFrame(values[v]).T
        df_aux = df_aux[['id','banda_num','banda_nombre', 'angulo_incidencia','valor_promedio', 'valor_desvioest']]
        df_aux['fecha'] = get_date(img_complete,sensor_name_sat, saocom_dates_xmet = saocom_dates_xmet)
        #-------------------------
        #print(d)
        df_aux = add_table_values_to_df(d,df_aux)
        #-----------------------------
        df_aux['sensor'] = sensor_name_sat
        df_aux['tipo_de_sensor'] = tipo_de_sensor
        df_aux['level'] = level
        df_aux['shp_file'] = shp_file
        df_aux['img_file'] = img_complete
        df_aux = df_aux.fillna(value='0')
        df_aux.to_csv(dataframes_folder+sensor_name_sat+'.csv', mode='a', header=False)
  return lista_de_imagenes_no_procesadas
        


In [19]:
# Inputs
# Corregir los paths de acuerdo al disco en el que se corre el script

cols = ['id', 'banda_num', 'banda_nombre', 'angulo_incidencia', 'valor_promedio', 'valor_desvioest', 'fecha', 'UP','transecta',
'geomorfo','cobertura', 'tipo_humed', 'posicion','perimetro','area','per_area','y_cen','x_cen','sensor','tipo_sensor','tipo_escena',
'shp_file','img_file']

# Direccion a la carpeta Raster:
#folder = '/media/nmorandeira/Atlas/_PROSAT_backup20220604/Raster/'#'gdrive/MyDrive/Raster/'
folder = '/media/nmorandeira/T7/Img_preprocesadas/Raster/'

# Direccion a la carpeta donde guardaremos los dataframes:
dataframes_folder = '/media/nmorandeira/T7/ExtraccionSAR/dataframes/'
# Direccion a la carpeta que contiene los shapefiles
shp_files_list = glob.glob('/media/nmorandeira/T7/ExtraccionSAR/ROIs/*shp')
nodatavalue=-9999
## Lista que guardara las imagenes que no fueron procesadas porque la informacion de bandas.txt no coincidia con el tif
total_de_imagenes_no_procesadas = []

In [ ]:
# Comienza el cálculo
# Obtiene las subfolders de Raster, en este caso la direccion a la carpea SAR
subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]

for s in subfolders:
  tipo_de_sensor =  os.path.basename(os.path.normpath(s)).upper()
  #sensorsfolders = [ f.path for f in os.scandir(s) if f.is_dir()]
  sensorsfolders = ['/media/nmorandeira/T7/ExtraccionSAR/SAOCOM_shifted/Raster/SAR/SAOCOM']
  #sensorsfolders = ['/media/nmorandeira/T7/Img_preprocesadas/Raster/SAR/ALOS']
  #sensorsfolders = ['/media/nmorandeira/Atlas/_PROSAT_backup20220604/Raster/SAR/SENTINEL']

  for sensor in sensorsfolders:
    sensor_name_sat =  os.path.basename(os.path.normpath(sensor)).upper()
    sensorandlevelfolders = [ f.path for f in os.scandir(sensor) if f.is_dir()]

    df = pd.DataFrame(columns=cols)
    df.to_csv(dataframes_folder+sensor_name_sat+'.csv')

    for shp_file in shp_files_list:
      d = vector_table_to_dic(shp_file)
      #sensorandlevelfolders = ['/media/nmorandeira/Atlas/_PROSAT_backup20220604/Raster/SAR/SAOCOM/SAOCOM_StripMap_QP']
      for sensor_and_level in sensorandlevelfolders:
        # SAOCOM tiene mas carpetas: polarizaciones y descomposiciones
        if sensor_name_sat == 'SAOCOM':
          for fold in [ f.path for f in os.scandir(sensor_and_level) if f.is_dir()]:
            lista_de_imagenes_no_procesadas = get_csv(fold, shp_file, dataframes_folder, sensor_name_sat)
            total_de_imagenes_no_procesadas.append(lista_de_imagenes_no_procesadas)
        else:
          lista_de_imagenes_no_procesadas = get_csv(sensor_and_level, shp_file, dataframes_folder, sensor_name_sat)
          total_de_imagenes_no_procesadas.append(lista_de_imagenes_no_procesadas)



---------------------------------------
Corriendo:  /media/nmorandeira/T7/ExtraccionSAR/SAOCOM_shifted/Raster/SAR/SAOCOM/SAOCOM_StripMap_QP/decomp/S1A_OPER_SAR_EOSSP__CORE_L1A_OLF_20220421T203040_shifted_x6y-3.tif


Warning 1: /media/nmorandeira/T7/ExtraccionSAR/SAOCOM_shifted/Raster/SAR/SAOCOM/SAOCOM_StripMap_QP/decomp/S1A_OPER_SAR_EOSSP__CORE_L1A_OLF_20220421T203040_shifted_x6y-3.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /media/nmorandeira/T7/ExtraccionSAR/SAOCOM_shifted/Raster/SAR/SAOCOM/SAOCOM_StripMap_QP/decomp/S1A_OPER_SAR_EOSSP__CORE_L1A_OLF_20220421T203040_shifted_x6y-3.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
